# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."  
We will show how to use them here, in the context of pinkfish.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.3f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [4]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from cache, if available.

In [5]:
ts = pf.fetch_timeseries(symbol)
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2021-02-23,388.950,380.200,384.660,387.500,107284100.000,387.500
2021-02-24,392.230,385.270,386.330,391.770,72433900.000,391.770
2021-02-25,391.880,380.780,390.410,382.330,146670500.000,382.330
2021-02-26,385.580,378.230,384.350,380.360,152534900.000,380.360
2021-03-01,390.920,380.572,385.590,390.115,75353922.000,390.115


Select timeseries between start and end.

In [6]:
ts = pf.select_tradeperiod(ts, start, end)
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.895
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,210.138
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.971
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.722
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,210.027


### Get info about TA-Lib

In [7]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [8]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [9]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [10]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [11]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [12]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [13]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [14]:
sma = SMA(ts)
sma.tail()

date
2021-02-23   383.968
2021-02-24   384.404
2021-02-25   384.523
2021-02-26   384.542
2021-03-01   384.931
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [15]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2021-02-23   342.704
2021-02-24   343.201
2021-02-25   343.650
2021-02-26   344.118
2021-03-01   344.661
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [16]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2021-02-23   342.727
2021-02-24   343.203
2021-02-25   343.704
2021-02-26   344.156
2021-03-01   344.654
dtype: float64

### Add a technical indicator to a pinkfish timeseries

In [17]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2021-02-23,388.950,380.200,384.660,387.500,107284100.000,387.500,342.727
2021-02-24,392.230,385.270,386.330,391.770,72433900.000,391.770,343.203
2021-02-25,391.880,380.780,390.410,382.330,146670500.000,382.330,343.704
2021-02-26,385.580,378.230,384.350,380.360,152534900.000,380.360,344.156
2021-03-01,390.920,380.572,385.590,390.115,75353922.000,390.115,344.654


### Try another one

Commodity Channel Index

In [18]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [19]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [20]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [21]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci
date,,,,,,,,
2021-02-23,388.950,380.200,384.660,387.500,107284100.000,387.500,342.727,-92.479
2021-02-24,392.230,385.270,386.330,391.770,72433900.000,391.770,343.203,12.879
2021-02-25,391.880,380.780,390.410,382.330,146670500.000,382.330,343.704,-173.196
2021-02-26,385.580,378.230,384.350,380.360,152534900.000,380.360,344.156,-218.207
2021-03-01,390.920,380.572,385.590,390.115,75353922.000,390.115,344.654,-43.960


### Now for something a little more difficult

Bollinger Bands

In [22]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [23]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [24]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [25]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [26]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [27]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [28]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower
date,,,,,,,,,,,
2021-02-23,388.950,380.200,384.660,387.500,107284100.000,387.500,342.727,-92.479,399.501,386.539,373.576
2021-02-24,392.230,385.270,386.330,391.770,72433900.000,391.770,343.203,12.879,400.231,387.037,373.843
2021-02-25,391.880,380.780,390.410,382.330,146670500.000,382.330,343.704,-173.196,398.797,386.589,374.380
2021-02-26,385.580,378.230,384.350,380.360,152534900.000,380.360,344.156,-218.207,397.861,385.995,374.130
2021-03-01,390.920,380.572,385.590,390.115,75353922.000,390.115,344.654,-43.960,395.581,386.388,377.195


In [29]:
print(MOM)

MOM([input_arrays], [timeperiod=10])

Momentum (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 10
Outputs:
    real


In [30]:
mom10 = MOM(ts, timeperiod=10)
mom10.tail()

date
2021-02-23    -3.010
2021-02-24     1.520
2021-02-25    -7.750
2021-02-26   -10.350
2021-03-01    -2.525
dtype: float64

In [31]:
ts['mom10'] = mom10
ts.head(50)

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower,mom10
date,,,,,,,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.895,nan,nan,nan,nan,nan,nan
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,210.138,nan,nan,nan,nan,nan,nan
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.971,nan,nan,nan,nan,nan,nan
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.722,nan,nan,nan,nan,nan,nan
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,210.027,nan,nan,nan,nan,nan,nan
2017-01-10,227.450,226.010,226.480,226.460,63771900.000,210.027,nan,nan,nan,nan,nan,nan
2017-01-11,227.100,225.590,226.360,227.100,74650000.000,210.620,nan,nan,nan,nan,nan,nan
2017-01-12,226.750,224.960,226.500,226.530,72113200.000,210.092,nan,nan,nan,nan,nan,nan
2017-01-13,227.400,226.690,226.730,227.050,62717900.000,210.574,nan,nan,nan,nan,nan,nan


In [32]:
m1 = ts['close'].pct_change(periods=10)
ts['m1'] = m1
ts.head(50)

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower,mom10,m1
date,,,,,,,,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.895,nan,nan,nan,nan,nan,nan,nan
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,210.138,nan,nan,nan,nan,nan,nan,nan
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.971,nan,nan,nan,nan,nan,nan,nan
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.722,nan,nan,nan,nan,nan,nan,nan
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,210.027,nan,nan,nan,nan,nan,nan,nan
2017-01-10,227.450,226.010,226.480,226.460,63771900.000,210.027,nan,nan,nan,nan,nan,nan,nan
2017-01-11,227.100,225.590,226.360,227.100,74650000.000,210.620,nan,nan,nan,nan,nan,nan,nan
2017-01-12,226.750,224.960,226.500,226.530,72113200.000,210.092,nan,nan,nan,nan,nan,nan,nan
2017-01-13,227.400,226.690,226.730,227.050,62717900.000,210.574,nan,nan,nan,nan,nan,nan,nan
